# API Testing

In [35]:
%load_ext autoreload
%autoreload 2

import requests
from sodapy import Socrata

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
#!/usr/bin/env python

#!/usr/bin/env python

# make sure to install these packages before running:
# pip install pandas
# pip install sodapy

import pandas as pd
from sodapy import Socrata

# Unauthenticated client only works with public data sets. Note 'None'
# in place of application token, and no username or password:
client = Socrata("data.cityofnewyork.us", key_token)

# Example authenticated client (needed for non-public datasets):
# client = Socrata(data.cityofnewyork.us,
#                  MyAppToken,
#                  username="user@example.com",
#                  password="AFakePassword")

# First 2000 results, returned as JSON from API / converted to Python list of
# dictionaries by sodapy.
results = client.get("43nn-pn8j", limit=500_000)

# Convert to pandas DataFrame
results_df = pd.DataFrame.from_records(results)

In [44]:
ny = results_df.copy()

In [45]:
ny

,camis,dba,boro,building,street,zipcode,phone,inspection_date,critical_flag,record_date,...,bbl,nta,cuisine_description,action,violation_code,violation_description,score,grade,grade_date,inspection_type
0,50106756,UNGARO COAL FIRED PIZZA CAFE,Staten Island,1298,FOREST AVENUE,10302,6464690930,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,...,5003870033,SI07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,50105716,STELLA'S,Brooklyn,559,5 AVENUE,11215,4155703174,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,...,3010480009,BK37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,41168748,DUNKIN,Bronx,880,GARRISON AVENUE,10474,7188614171,2022-03-30T00:00:00.000,Not Critical,2023-10-26T06:00:11.000,...,2027390001,BX27,Donuts,Violations were cited in the following area(s).,10J,Hand wash sign not posted,13,A,2022-03-30T00:00:00.000,Cycle Inspection / Initial Inspection
3,50131566,EXTACY LOUNGE,Queens,7701,JAMAICA AVE,11421,3478752367,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,...,4088410152,QN53,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,50128764,RUNNING KIDS,Brooklyn,856,64 STREET,11220,7188338856,1900-01-01T00:00:00.000,Not Applicable,2023-10-26T06:00:14.000,...,3057420027,BK34,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207924,41571471,LITTLE CAESARS,Queens,14605,JAMAICA AVENUE,11435,7182973450,2022-10-21T00:00:00.000,Not Critical,2023-10-26T06:00:11.000,...,4096760044,QN61,Pizza,Violations were cited in the following area(s).,09B,Thawing procedure improper.,7,A,2022-10-21T00:00:00.000,Cycle Inspection / Initial Inspection
207925,50130884,NORA PIZZA,Bronx,2456,EASTCHESTER ROAD,10469,6464921148,2023-05-15T00:00:00.000,Critical,2023-10-26T06:00:11.000,...,2044780032,BX31,Pizza,Violations were cited in the following area(s).,02G,Cold TCS food item held above 41 °F; smoked or...,26,Z,2023-05-15T00:00:00.000,Pre-permit (Operational) / Re-inspection
207926,41139255,HUDSON FOOD COURT,Manhattan,333,HUDSON STREET,10013,2123672192,2022-09-06T00:00:00.000,Not Critical,2023-10-26T06:00:11.000,...,1005970055,MN24,American,Violations were cited in the following area(s).,08A,Establishment is not free of harborage or cond...,0,A,2022-09-06T00:00:00.000,Cycle Inspection / Re-inspection
207927,50011932,ANTICA RISTORANTE,Manhattan,8,STONE STREET,10004,2124803880,2019-11-25T00:00:00.000,Critical,2023-10-26T06:00:11.000,...,1000100033,MN25,Italian,Violations were cited in the following area(s).,04M,Live roaches present in facility's food and/or...,48,C,2019-11-25T00:00:00.000,Cycle Inspection / Re-inspection
